# Graded Challenge 3

`DESTI RATNA KOMALA  (RMT-020)`

## 1. Pendahuluan

### 1.1 Judul


**"EVALUASI PERILAKU JUMLAH PENJUALAN TERHADAP PERIODE PERAYAAN MASAL PADA PERUSAHAAN THELOOK DI MARS UNTUK TAHUN 2022"**



### 1.2 Latar Belakang

TheLook adalah salah satu perusahaan e-commerce terbesar di Mars yang menjual produk *fashion* untuk seluruh kalangan baik wanita, pria, style kasual maupun formal.
smart:

Perusahaan TheLook ingin meningkatkan penjualan pada musim perayaan atau periode Q4. Hal tersebut dilakukan agar perusahaan TheLook dapat tumbuh dan berkembang menjadi lebih besar. Hal tersebut dilakukan dengan mencari banyak investor. Kenaikan total pendapatan per bulan sebesar 5% dapat menjadi track record yang sangat bagus bagi perubahan TheLook untuk menggait investor.


### 1.3 Rumusan Masalah

Perusahaan TheLook ingin meningkatkan penjualan pada musim perayaan dan bulan menjelang tahun baru atau pada kuartal 4. Namun perusahaan TheLook tidak mengetahui kategori produk mana yang volume produksinya harus ditingkatkan agar jumlah pendapatan dapat meningkat.

Oleh karena itu, perusahaan akan mencari kategori produk tertinggi/terbanyak yang dibeli oleh pelanggan TheLook pada tiap bulannya untuk periode Q1 hingga Q3, untuk dijadikan acuan pada bulan-bulan selanjutnya. Volume produk pada kategori tersebut akan ditingkatkan jumlah produksinya untuk meningkatkan pendapatan pada perushaan TheLook



## 2. Overview Dataset


### 2.1. Data dan Label pada`thelook_ecommerce`

Dataset yang digunakan pada analisis ini adalah data `thelook__ecommerce` yang terdiri sub data berupa:

<justify>

|Sub Data thelook_ecommerce|Definisi|Jumlah (Kolom, baris)|
|-|---|---|
|users|seseorang yang memesan/membeli barang|(100000, 15)
|orders|Permintaan pembelian barang dari pembeli|(125103, 9)
|order_items|Barang-barang yang dipesan oleh pembeli|(181634, 11)
|products|Sesuatu yang ditawarkan pada produsen|(29120, 9)
|events|Semua laporan pemesanan, kerusakan, pengembalian,|(2428542, 13)|
||pengiriman, dll yang berhubungan dengan produk||
|inventory_items|Semua barang yang dimiliki oleh perusahaan |(490549, 12)
||untuk dijual kepada pelanggan||
|distribution_centers|Tempat pengumpulan dan penyimpanan barang agar dapat|(10, 4)
||didistribusikan ke banyak retail dan grosir dalam satu waktu||


Urutan proses dari users hingga ke distribution_centers

Users ---> orders---> order_items--->  products ---> events---> inventory_items ---> distribution_centers

### 2.2 Quarter Per tahun

Berikut adalah informasi mengenai pembagian bulan berdasarkan quartan masing-masing.

Tabel 2.2 quarter per tahun

|Quartal |periode|
|-----|----|
|Q1|1 Januari - 31 Maret|
|Q2|1 April -30 Juni|
|Q3|1 Juli - 30 September|
|Q4|1 Oktober - 31 Desember|



---



---



## 3. Ekplorasi Data

### 3.1 Menghubungkan Google colab dengan BigQuery SQL
Sebelum dilakukan pengolahan data, data akan diakses menggunakan `bigquery-public-data` pada Google Cloud Big Query. Data yang digunakan berasal dari database `thelook_ecommerce` yang akan dikoneksikan pada`BigQuery` paa project id p0-gc3-389907. Berikut adalah proses pengambilan data ke google colab.


In [1]:
#hubungkan google.colab dengan bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
#Import data dari bigquery ke google.cloud
from google.cloud import bigquery

In [3]:
#project id yang ingin diimport adalah "p0-gc3-389907"
project_id = "p0-gc3-389907"
client = bigquery.Client(project=project_id)

Project_id pada bigquery pada analisis ini adalah `"p0-gc3-389907"` dan berdasarkan pemanggilan diatas, data tersebut berhasil terhubung dengan google colab.



---



### 3.2 Jumlah Transaksi berstatus *Complete* tiap bulan selama Q1 sampai Q3 di tahun 2022 (01 Januari 2022 - 30 September 2022)


Sebelum mendefinisikan kriteria kesuksesan, perlu dilakukan pencarian jumlah transaksi pada data e-commerce sebagai berikut.

In [4]:
#Mencari Jumlah transaksi yang berstatus complete
No1 = client.query('''
SELECT
  extract(year from OI.created_at) as Tahun,
  extract(month from OI.created_at) as Bulan,
  count(OI.order_id) as Jumlah_Transaksi,
FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as O
ON OI.user_id=O.user_id
WHERE O.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
GROUP BY Tahun, Bulan
ORDER BY Tahun,Bulan ASC;
''').to_dataframe()

In [5]:
#Melihat berapa jumlah baris dan kolom pada No1
No1.shape

(9, 3)

In [6]:
#Tampilkan data pada No1
No1

,Tahun,Bulan,Jumlah_Transaksi
0,2022,1,1981
1,2022,2,1863
2,2022,3,2206
3,2022,4,2131
4,2022,5,2427
5,2022,6,2547
6,2022,7,2820
7,2022,8,2785
8,2022,9,2840


Berdasarkan data tersebut, jumlah transaksi yang berstatus `Complete` tiap bulannya selama Q1 hingga Q3 (01 Januari 2022 - 30 September 2022):
* Terjadi peningkatan jumlah transaksi tiap bulannya
* Meskipun demikian terlihat bahwa fluktuasi tidak tentu tetap terjadi tiap bulannya.
* Pada bulan Januari ke Febuari terjadi penurunan secara drastis dari 1981 menjadi 1863 jumlah transaksi. Hal ini dipengaruhi oleh para pelanggan sudah membeli seluruh produk yang mereka inginkan pada bulan-bulan perayaan yaitu pada bulan Desember dan Januari seperit perayaan natal dan tahun baru.





---



### 3.3 Total Penjualan berstatus *Complete* tiap bulan selama Q1 sampai Q3 di tahun 2022 (01 Januari 2022 - 30 September 2022)


In [7]:
#Mencari Total penjualan tiap bulan selama Q1 hingga Q3
No2 = client.query('''
SELECT
  extract(year from OI.created_at) as Tahun,
  extract(month from OI.created_at) as Bulan,
  SUM(OI.sale_price * O.num_of_item) as Jumlah_Penjualan_USD,
FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as O
ON OI.user_id=O.user_id
WHERE O.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
GROUP BY Tahun, Bulan
ORDER BY Tahun,Bulan ASC;
''').to_dataframe()

In [8]:
#Melihat berapa jumlah baris dan kolom pada No1
No2.shape

(9, 3)

In [9]:
#Tampilkan data pada No2
No2

,Tahun,Bulan,Jumlah_Penjualan_USD
0,2022,1,205187.140244
1,2022,2,192272.470221
2,2022,3,222751.000460
3,2022,4,204026.470322
4,2022,5,230494.610058
5,2022,6,259981.100263
6,2022,7,277688.790208
7,2022,8,277813.770223
8,2022,9,273534.490132


Jumlah penjualan yang berstatus `Complete` tiap bulan selama Q1 hingga Q3 (01 Januari 2022 - 30 September 2022):
* Jumlah penjualan mengalami peningkatan tiap bulannya, namun fluktuasi tidak tentu tetap terjadi.
* Pada bulan Maret ke April terjadi penurunan total penjualan yang sangat signifikan yaitu dari `$222.751` menjadi `$204.026` atau kehilangan sekitar `$18.545` dari bulan sebelumnya, meskipun total transaksi `complete` tidak mengalami penurunan, melainkan peningkatan.
* Hal ini diperkirakan terjadi karena tidak ada perayaan masal di sekitar bulan April dan kebutuhan telah terpenuhi di bulan Maret.





---



### 3.4 Jumlah Users berstatus *Complete* tiap bulan selama Q1 sampai Q3 di tahun 2022 (01 Januari 2022 - 30 September 2022)


In [10]:
#Mencari jumlah users yang melakukan transaksi pada Q1 hingga Q3
No3 = client.query('''
SELECT
  extract(year from OI.created_at) as Tahun,
  extract(month from OI.created_at) as Bulan,
  count(DISTINCT U.id) as Users,
FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` as U
ON OI.user_id=U.id
WHERE OI.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
GROUP BY Tahun, Bulan
ORDER BY Tahun,Bulan ASC;
''').to_dataframe()
No3

,Tahun,Bulan,Users
0,2022,1,649
1,2022,2,620
2,2022,3,767
3,2022,4,745
4,2022,5,825
5,2022,6,890
6,2022,7,920
7,2022,8,978
8,2022,9,1029


Jumlah Users yang berstatus `Complete` tiap bulan selama Q1 hingga Q3 (01 Januari 2022 - 30 September 2022):
* Jumlah Users mengalami peningkatan tiap bulannya, namun fluktuasi tidak tentu tetap terjadi.
* Pada bulan Febuari, jumlah Users mengalami penurunan yang cukup signifikan sebesar 86 Users.
* Hal ini sejalan dengan penurunan jumlah transaksi pada bulan Febuari dan dikarenakan kebutuhan yang diinginkan pengguna telah dibeli pada bulan-bulan perayaan seperti Desember dan Januari



---



### 3.5 Kategori Produk Terbanyak berstatus *Complete* tiap bulan selama Q1 sampai Q3 di tahun 2022 (01 Januari 2022 - 30 September 2022)

In [11]:
#Mencari kategori produk terbanyak tiap bulan
No4 = client.query('''
    SELECT Bulan, kategori, Jumlah_Item
    FROM(
        SELECT
            x.*,
            ROW_NUMBER() OVER(PARTITION BY Bulan ORDER BY Jumlah_Item desc) rn

          FROM (
                SELECT
                extract(year from OI.created_at) as Tahun,
                extract(month from OI.created_at) as Bulan,
                P.category as kategori,
                sum(O.num_of_item) as Jumlah_Item
              FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
                LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as O
                ON OI.order_id=O.order_id
                LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` as P
                ON OI.product_id=P.id
              WHERE O.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
              GROUP BY Tahun, Bulan, kategori
          ) x
          ORDER BY Bulan
        )y
    WHERE rn =1
''').to_dataframe()
No4

,Bulan,kategori,Jumlah_Item
0,1,Tops & Tees,136
1,2,Tops & Tees,132
2,3,Shorts,146
3,4,Intimates,171
4,5,Fashion Hoodies & Sweatshirts,152
5,6,Shorts,191
6,7,Intimates,217
7,8,Intimates,233
8,9,Fashion Hoodies & Sweatshirts,192


Kategori produk dengan pembelian tertinggi dan memiliki status `Complete` tiap bulan selama Q1 hingga Q3 (01 Januari 2022 - 30 September 2022):
* kategori produk yang paling sering dibeli selama Q1 samapai Q3 yaitu intimates dan jeans.
* Pada bulan Agustu kategori tertinggi adalah intimates. Sementara itu, terjadi penurunan untuk bulan April ke bulan Mei dari 171 menjadi 152.
* selain itu, intimates merupakan kategori yang mendominasi pada Q1 hingga Q3, hal ini menunjukkan bahwa kebanyakan dari pembeli adalah orang-orang dewasa baik pria maupun wanita.
* kategori sweatshirts, hoddies, tops dan shorts juga secara bersamaan menjadi peringkat kedua menjajahi hati pelanggan. Ini berarti kebanyakan dari pelanggan TheLook menyukai pakaian dengan kategori yang nyaman dan casual.






---



### 3.6 Kategori Produk yang Paling Banyak mendapatkan Pendapatan dan *Complete* tiap bulan selama Q1 sampai Q3 di tahun 2022 (01 Januari 2022 - 30 September 2022)

In [12]:
#Mencari kategori produk yang paling banyak mendapatkan pendapatan
No5 = client.query('''
    SELECT Bulan, kategori, Jumlah_Item, Jumlah_Penjualan_USD
    FROM(
        SELECT
            x.*,
            ROW_NUMBER() OVER(PARTITION BY Bulan ORDER BY Jumlah_Item desc) rn

          FROM (
                SELECT
                extract(year from OI.created_at) as Tahun,
                extract(month from OI.created_at) as Bulan,
                P.category as kategori,
                sum(O.num_of_item) as Jumlah_Item,
                SUM(OI.sale_price * O.num_of_item) as Jumlah_Penjualan_USD
              FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
                LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as O
                ON OI.order_id=O.order_id
                LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` as P
                ON OI.product_id=P.id
              WHERE O.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
              GROUP BY Tahun, Bulan, kategori
          ) x
          ORDER BY Bulan
        )y
    WHERE rn =1
''').to_dataframe()
No5

,Bulan,kategori,Jumlah_Item,Jumlah_Penjualan_USD
0,1,Tops & Tees,136,5402.539999
1,2,Tops & Tees,132,5743.620000
2,3,Shorts,146,6476.160021
3,4,Intimates,171,5050.710000
4,5,Fashion Hoodies & Sweatshirts,152,8397.289980
5,6,Shorts,191,9479.900016
6,7,Intimates,217,6800.690009
7,8,Intimates,233,7195.790015
8,9,Fashion Hoodies & Sweatshirts,192,10154.719980


Kategori produk dengan total penjualan tertinggi dan memiliki status `Complete` tiap bulan selama Q1 hingga Q3 (01 Januari 2022 - 30 September 2022):
* Kategori produk paling tinggi adalah jeans pada bulan september dengan total penjualan sebesar `$10.154` dan paling rendah adalah kategori intimates pada bulan April dengan total penjualan sebesar `S5.050`
* pada bulan September kategori produk Fashion hoodies dan sweatshirts sangat tinggi yang kemungkinan dikarena oleh banyaknya orang yang ingin mengganti hoodies lama mereka yang mereka beli pada saat perayaan tahun baru dan ingin membeli yang baru pada bulan tersebut.
* Selain itu, bulan September mendekati periode akhir tahun, sehingga orang-orang banyak membeli hoodies pada bulan tersebut untuk persiapan musim dingin.



---



### 3.7 Korelasi jumlah transaksi dan total penjualan terhadap users

In [13]:
#Mencari Korelasi jumlah transaksi dan total penjualan terhadap users.
No6 = client.query('''
SELECT x.Tahun as Tahun_Q1_Q3,
  corr(x.Jumlah_Penjualan_USD, x.Users) as Korelasi_Jumlah_Penjualan_Users,
  corr(x.Jumlah_Transaksi, x.Users) as Korelasi_Jumlah_Transaksi_Users,
FROM (
      SELECT
      extract(year from OI.created_at) as Tahun,
      extract(month from OI.created_at) as Bulan,
      count(DISTINCT OI.user_id) as Users,
      COUNT(Distinct OI.order_id) AS Jumlah_Transaksi,
      SUM(OI.sale_price * O.num_of_item) as Jumlah_Penjualan_USD,

      FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
      LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as O
      ON OI.user_id=O.user_id
      LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` as U
      ON OI.user_id=U.id
      WHERE OI.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
      GROUP BY Tahun, Bulan
    ) x
 GROUP BY 1
 ORDER BY 1

''').to_dataframe()
No6


,Tahun_Q1_Q3,Korelasi_Jumlah_Penjualan_Users,Korelasi_Jumlah_Transaksi_Users
0,2022,0.938347,0.999889


Korelasi antara jumlah transaksi dengan jumlah penjualan terhadap users yang memiliki status `Complete`:
* Berdasarkan data tersebut, korelasi antara users dengan jumlah_transaksi lebih dekat dibandingkan dengan jumlah penjualannya.
* Hal ini berarti semakin banyak jumlah Usersnya maka semakin banyak pula  jumlah transaksinya yang dilakukan.
* Korelasi kedua variabel tersebut (Jumlah penjualan dan Jumlah transaksi) terhadap Users dapat dikatakan hampir sama, yang masing-masing bernilai 0.938347 dan 0.999889.
* Selain itu, kedua koefiesien tersebut bernilai positif yang berarti kedua variable tersebut cenderung mengalami peningkatan sebanding dengan banyaknya users.

### 3.8 Persentase Peningkatan/Penurunan Jumlah Penjualan

In [14]:
#Mencari Persentase Peningkatan/Penurunan Jumlah Penjualan
Persen_Grown = client.query('''
    SELECT Bulan,  Jumlah_Penjualan_USD, persen_grown
    FROM(
        SELECT
            x.*,
            100.0 * (1 - LAG(Jumlah_Penjualan_USD) OVER (ORDER BY Bulan) / Jumlah_Penjualan_USD) AS persen_grown
          FROM (
                SELECT
                  extract(year from OI.created_at) as Tahun,
                  extract(month from OI.created_at) as Bulan,
                  SUM(OI.sale_price * O.num_of_item) as Jumlah_Penjualan_USD,
                  FROM `bigquery-public-data.thelook_ecommerce.order_items` as OI
                  LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as O
                  ON OI.user_id=O.user_id
                  JOIN `bigquery-public-data.thelook_ecommerce.products` as P
                  ON OI.product_id=P.id
                  WHERE O.status='Complete' and OI.created_at BETWEEN "2022-01-01" AND "2022-09-30"
                  GROUP BY Tahun, Bulan
          ) x
        )y
        ORDER BY 1
''').to_dataframe()
Persen_Grown

,Bulan,Jumlah_Penjualan_USD,persen_grown
0,1,205187.140244,NaN
1,2,192272.470221,-6.716859
2,3,222751.000460,13.682780
3,4,204026.470322,-9.177500
4,5,230494.610058,11.483192
5,6,259981.100263,11.341782
6,7,277688.790208,6.376811
7,8,277813.770223,0.044987
8,9,273534.490132,-1.564439


 Persentase penurunan terendah pada perusahaan TheLook di kuartal 1 hingga kuarta 3 tahun 2022 adalah sebesar `~9%` pada bulan Mei dan persentase peningkatan tertinggi adalah sebesar `~13%` pada bulan Maret. Berdasarkan data tersebut, persentase jumlah penjualan untuk perusahaan TheLook tidak meningkat secara konsisten sebanyak 5% seperti target yang diingikan.

## 4. Analisis



&emsp;&emsp; Berdasarkan data yang telah diolah dapat disimpulkan bahwa tiap bulan terdapat kategori produk masing-masing yang dapat digunakan sebagai acuan dalam meningkat penjualan untuk tahun berikutnya atau tahun 2023. Salah satu yang paling sering dibeli adalah produk intimates, yang disusul oleh hoodies, sweatshirs dan shorts. Sehingga, untuk kategori-kategori tersebut akan ditingkatkan volume produksinya.



&emsp;&emsp;Meskipun demikian, target untuk meningkatkan jumlah penjualan perusahaan sebesar 5%  tiap bulan pada periode Q1 hingga Q3 tidak dapat dipenuhi. Karena berdasarkan persen peningkatan dari jumlah penjualan perbulan, peningkatan dan penurunan mengalami fluktuasi yang tidak tentu.


&emsp;&emsp;Apabila CEO TheLook ingin menargetkan pendapatan di awal kuartal 4 harus mencapai `$250.000`, maka hal tersebut dapat dikatakan masuk akal. Alasan utamanya adalah pada kuarta 4, periode tersebut merupakan periode dimana banyak perayaan masal terjadi seperti natal, dan tahun baru. Pelanggan akan membeli lebih banyak barang untuk perayaan-perayaan penting tersebut.


&emsp;&emsp;Jika didasarkan dari kuarta 1 hingga kuartal 3, persen penurunan paling signifikan yaitu sebesar `~9%`. Sementara di 3 bulan terakhir pada kuartal 3, nilai jumlah pendapatan sangat stabil di kisaran `~$270.000`. Sehingga jika terjadi penurunan signifikan terburuk sebesar `~9%` dari `~$270.000` maka pendapatan pada kuartal 4 yaitu dikisaran `~$252.810`. Nilai tersebut masih berada `~$2.810` di atas jumlah pendapatan yang ditargetkan oleh CEO.


&emsp;&emsp;Sementara itu, jika terjadai keanikan signifikan `~13%`dari`~$270.000` maka perusahaan TheLook bisa mendapatkan jumlah pendapatan hingga dikisaran `~$305 100`. Nilai perkiraan kenaikan dan penurunan jumlah pendapatan ini belum termasuk faktor perayaan masal/serentak di kuartal 4 yang memungkinkan pelanggan untuk membeli pakaian/produk-produk baru untuk perayaan natal dan tahun baru.


&emsp;&emsp;Sehingga dapat dipastikan bahwa target dari CEO TheLook masuk akal dan dapat tercapai. Dan keuntungan untuk kuartal 4 akan meningkat lebih tinggi lagi karena faktor kebutuhan yang meningkat pada periode perayaan masal.

